<a href="https://colab.research.google.com/github/pietroventurini/machine-learning-notes/blob/main/6.2%20-%20Optimization%20Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimization Algorithms

## Contents

1. [**Prerequisites**](#Prerequisites)  
    1. [Exponentially Weighted Average](#Exponentially-Weighted-Average)
2. [**Optimization Algorithms**](#Optimization-Algorithms)  
    1. [Momentum](#Momentum)  
    2. [Nesterov Accelerated Gradient](#Nesterov-Accelerated-Gradient)  
    3. [RMSProp](#RMSProp)  
    4. [Adam](#Adam) 
3. [**Common Practices**](#Common-Practices)
    1. [Dynamic learning rate](#Dynamic-learning-rate)  
    2. [Tuning hyperparameters](#Tuning-hyperparameters)  

<a name='Prerequisites'></a>
# Prerequisites

In order to understand the optimization algorithms that we'll present, we need first to understand how exponentially weighted average works.

<a name='Exponentially-Weighted-Average'></a>
## Exponentially Weighted Average

Consider a set of data $\left\{x_1,\dots, x_m \right\}$ describing the daily temperature in a city over a time span. The time series may look a little bit noisy, so, in order to highlight the trends, we could compute an exponentially weighted moving average, which can be done using the following recursive formulation:

$$v_t = \beta v_{t-1} + (1-\beta)x_t$$

which can be initialized with $v_0=0$. We can think at the term $v_t$ as approximately averaging over $\frac{1}{1-\beta}$ days temperature, for instance, if $\beta=0.9$, that is like averaging over the last 10 days temperature. If we average over a larger window (e.g. $\beta=0.98$, which corresponds approximatively to 50 days), the exponentially weighted average adapts more slowly to changes in the data, and that is because, when $\beta$ is very close to one, we are giving more importance to $v_{t-1}$ rather than to the current data $x_t$.

The recursive formulation is an efficient way to update the moving average, without the need to keep in memory all the previous data.

### Bias correction

Assume $\beta=0.98$ and consider what would happen with the first bunch of averages:

- $v_0 = 0$
- $v_1 = \beta v_0 + (1 - \beta)x_1 = 0.02 x_1$
- $v_2 = \beta v_1 + (1 - \beta)x_2 = 0.98\cdot 0.02 x_1 + 0.02 x_2 = 0.0196x_1 + 0.02x_2$

We can see that, in the moment we combine the first two measurements, their weighted average is far less than any of the two values (Since we are computing a linear combination weighted by very small coefficients). In order to adress this issue and compute a more accurate estimate, we can rescale $v_t$ by $\frac{1}{1-\beta^t}$. So, when $t=2$ we would have $1-\beta^2 = 1 - 0.98^2 = 0.0396$ and we would compute, instead of $v_2$:

$$\frac{v_2}{1-\beta^2} = \frac{0.0196x_1 + 0.02x_2}{0.0396}$$

As you might have noticed, the sum of the coefficients at the numerator is equal to the denominator. This operation, sometimes called _bias correction_ can help exponentially weighted moving average being more accurate, especially with the initial estimates.

<a name='Optimization-Algorithms'></a>
# Optimization Algorithms

We can use variants of the stochastic or mini-batch gradient descent that may help to speed up learning and escape local minima. One of these techniques is the gradient descent with momentum. Two other popular optimization algorithms that have been shown to work well on a wide range of neural network architectures are RMSProp and Adam.

<a name='Momentum'></a>
## Momentum

There exist several heuristics that can be used to avoid getting stuck in local minima and may help accelerate the learning. For example we can include a **weight momentum** in the weight update. The basic idea is to compute an **exponentially weighted average** (governed by a parameter $\beta$) of the gradients and to use it to update our weights instead. Assume we are performing mini-batch gradient descent, and consider the update step for the $l$-th layer (we'll omit the superscript denoting the layer in the next lines). First we compute the derivatives $\mathrm{d}W$ and $\mathrm{d}\mathbf{b}$ of the loss function with respect to that layer's weights, on the current mini-batch. Then we update the weights in this way:

$$V_{\mathrm{d}W} = \beta V_{\mathrm{d}W} + (1-\beta) \mathrm{d}W$$

$$V_{\mathrm{d}b} = \beta V_{\mathrm{d}b} + (1-\beta) \mathrm{d}b$$

$$W = W - \alpha V_{\mathrm{d}W}$$

$$b = b - \alpha V_{\mathrm{d}b}$$

In this way we can smooth out the steps of gradient descent and follow a more straightforward path. Weight momentum can also be interpreted by making an analogy with physics: the current value of the weights represents the position of a physical object rolling down a surface (defined by the cost function). The term $\mathrm{d}W$ can be thought as an acceleration and the term $V_{\mathrm{d}W}$ as a velocity. The hyperparameter $\beta$, which is smaller than $1$, represents a _friction_ and prevents the object from speeding up without limit, but rather than taking each step independently from the previously taken ones, the object can gain momentum from and, eventually, escape a local minimum.

In practice $V_{\mathrm{d}W}$ is a matrix of the same dimension of $W$, $V_{\mathrm{d}b}$ is a vector of the same dimension of $b$, and they are both initialized to zero. Tipically, $\beta$ is initialized to some value around $0.9$.

<a name='Nesterov-Accelerated-Gradient'></a>
## Nesterov Accelerated Gradient

Nesterov Accelerated Gradient is slighlty different than momentum in the sense that we kind of "look into the future" to see how much momentum is required. The update equations become:

$$\delta^{(k+1)} = -\eta \nabla L(W^{(k)} + \alpha \delta^{(k)}) + \alpha\delta^{(k)}$$

$$W^{(k+1)} = W^{(k)} + \delta^{(k+1)}$$

The same holds also for $b$. With Nesterov momentum, first we move in the previous accumulated gradient computed the iteration before (from $W^{(k)}$ to $W^{(k)} + \alpha\delta^{(k)}$, then we compute the gradient in that point ($-\eta \nabla L(W^{(k)} + \alpha \delta^{(k)})$) and finally make a correction.

<img src="https://github.com/pietroventurini/machine-learning-notes/blob/main/images/neural_networks/Nesterov.png?raw=1" style="width:40em; display: block; margin-left: auto; margin-right: auto;" />

<a name='RMSProp'></a>
## RMSProp

Root Mean Square Propagation ([RMSProp](http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf)) is quite similar to weight momentum, but it divides the gradient by a running weighted average. The main reason behind this method is that the fact that the magnitude of the gradient for different weights may change during the learning makes it difficult to choose a single global learning rate.

$$S_{\mathrm{d}W} = \beta_2 S_{\mathrm{d}W} + (1-\beta_2) \mathrm{d}W^2$$

$$S_{\mathrm{d}b} = \beta_2 S_{\mathrm{d}b} + (1-\beta_2) \mathrm{d}b^2$$

$$W = W - \alpha \frac{\mathrm{d}W}{\sqrt{S_{\mathrm{d}W}}+\varepsilon}$$

$$b = b - \alpha \frac{\mathrm{d}b}{\sqrt{S_{\mathrm{d}b}}+\varepsilon}$$

Note that the squaring operation is computed element-wise, and we used different letters $S_{\mathrm{d}W}$ and $\beta_2$ than with momentum, because with Adam we are going to combine both momentum and RMSProp. $\varepsilon$ is a small term (in practice we can set it to $10^{-8}$) that prevents dividing by some quantity close to zero that would make the numerator explode. Note also that, differently from weight momentum, here we dump out the oscillations by dividing by a term which is proportional to the width of the step (to the derivative of the loss function with respect to $W$ or $b$), therefore, wider steps (steps in a direction in which the function is steeper) will be dump out more than narrow steps (steps in a direction in which the function is flatter). In other words, every parameter is weighted by a different learning rate.

A consequence of this is that we can use a larger learning rate $\alpha$ without diverging in the steepest directions.

<a name='Adam'></a>
## Adam

Adaptive moment estimation (Adam) puts together weight momentum and RMSProp. The parameters are initialized to zero.

$$V_{\mathrm{d}W}=0, \quad S_{\mathrm{d}W}=0, \quad V_{\mathrm{d}b}=0, \quad S_{\mathrm{d}b}=0.$$

Then, on iteration $t$:
- Compute $\mathrm{d}W$ and $\mathrm{d}b$ using the current mini-batch 
- Compute Momentum exponentially weighted average:  
    - $V_{\mathrm{d}W}=\beta_1 V_{\mathrm{d}W} + (1-\beta_1)\mathrm{d}W$  
    - $V_{\mathrm{d}b}=\beta_1 V_{\mathrm{d}b} + (1-\beta_1)\mathrm{d}b$  
- Compute the RMSProp update terms:
    - $S_{\mathrm{d}W}=\beta_2 S_{\mathrm{d}W} + (1-\beta_2)\mathrm{d}W^2$  
    - $S_{\mathrm{d}b}=\beta_2 S_{\mathrm{d}b} + (1-\beta_2)\mathrm{d}b^2$  
- Perform the bias correction:
    - $V_{\mathrm{d}W}^{corr} = \frac{V_{\mathrm{d}W}}{1-\beta_1^t}$  
    - $V_{\mathrm{d}b}^{corr} = \frac{V_{\mathrm{d}b}}{1-\beta_1^t}$  
    - $S_{\mathrm{d}W}^{corr} = \frac{S_{\mathrm{d}W}}{1-\beta_2^t}$  
    - $S_{\mathrm{d}W}^{corr} = \frac{S_{\mathrm{d}W}}{1-\beta_2^t}$  
- Update the weights:
    - $W := W - \alpha \frac{V_{\mathrm{d}W}^{corr}}{\sqrt{S_{\mathrm{d}W}^{corr}}+\varepsilon}$
    - $b := b - \alpha \frac{V_{\mathrm{d}b}^{corr}}{\sqrt{S_{\mathrm{d}b}^{corr}}+\varepsilon}$
    
This algorithm has some hyperparameters that have to be tuned, and others that are tipically initialized to some common values:
- Learning rate $\alpha$ needs to be tuned (we could try few values and choose the one yielding the best result, or adopt learning rate decay).
- $\beta_1 = 0.9$
- $\beta_2 = 0.999$
- $\varepsilon = 10^{-8}$

<a name='Common-practices'></a>
# Common practices

<a name='Dynamic-learning-rate'></a>
## Dynamic learning rate

Instead of using a fixed, chosen a priori, learning rate, $\alpha$ is often replaced by a learning rate that decreases over time, for example:

$$\alpha = \frac{\alpha_0}{1 + \text{decay_rate} \cdot \text{epoch_num}}$$

where $\alpha_0$ is the initial learning rate.

There exist other learning rate decay methods, for instance:

- **Exponential decay**:

$$\alpha = k^{\text{epoch_num}}\cdot \alpha_0$$

where $k$ is a constant, e.g. $k=0.95$.

- **Based on epoch number:**

$$\alpha = \frac{k}{\sqrt{\text{epoch_num}}} \cdot \alpha_0$$

where $k$ is a constant.

- **Based on batch size:**

$$\alpha = \frac{k}{\sqrt{\text{batch_size}}} \cdot \alpha_0$$

where $k$ is a constant.





<a name='Tuning-hyperparameters'></a>
## Tuning hyperparameters

When training neural networks we normally have to deal with many hyperparameters: learning rate, learning rate decay, number of epochs, dropout probability, hyperparameters that are specific to the optimization algorithm in use, number of layers, number of units for each hidden layer, batch size and so on... some hyperparameters are more important than others, for instance, in many applications, the most important one is considered to be the learning rate.

In the notebook about _"Model evaluation and validation"_ we have talked about grid search as a method to choose a combination of hyperparameters leading to a better model. When the number of hyperparameters is quite small, that would be a good approach, anyway, with deep learning, we typically don't know a-priori which hyperparameters are going to be the most important ones. Consider the following example: we want to tune the learning rate $\alpha$ and the hyperparameter $\varepsilon$ that appears in Adam when we perform the update step. If we constructed a grid of 25 points (left figure) using 5 different values for $\alpha$ respectively combined with 5 different values for $\varepsilon$, we would disccover that the perforance of our neural network are mainly influenced by $\alpha$ while $\epsilon$ doesn't play an important role in that. This means that we trained 25 different neural networks invain, when we could have obtained the same result with just 5 of them.

In order to cope with this issue, it is suggested to pick the values for the hyperparameters at random (right figure). In that case, even if different values of $\varepsilon$ doesn't make significant improvements, we are trying a larger set of values for $\alpha$.

<img src="https://github.com/pietroventurini/machine-learning-notes/blob/main/images/neural_networks/random_search.png?raw=1" style="width:40em; display: block; margin-left: auto; margin-right: auto;" />

We can iterate the search process by sampling hyperparameters in a coarse to fine scheme: at each iteration, we search in a neighborhood of the values that worked best at the previous iteration. In other words, we zoom in to a smaller region of the hyperparameter space and sample more densely within that region.

### Choosing an appropriate scale

For some hyperparameters it is reasonable to pick their values uniformily between a lower bound and an upper bound. For instance, when choosing the number of hidden units within a hidden layer, we can reasonably sample values uniformily between, say, 50 and 100. The same holds for the number of hidden layers, which can be manually set, for instance, to 2, 3 or 4.

Anyway, if we consider the learning rate $\alpha$, and we sample a bunch of values between 0.0001 and 1, we will end using 90% of the resources searching for values between 0.1 and 1, and just 10% of the resources for searching between 0.0001 and 0.1. A more suitable way would be to sample those values uniformily on the log scale. On that scale, the points 0.0001, 0.001, 0.01, 0.1 and 1 are equally spaced. In practice, that can be obtained in the following way:

```python
alpha = 10**np.random.uniform(-4, 0) # ⍺ ∈ [0.0001, 1]
```

This trick can be used, for instance, to sample values for the hyperparameter $\beta$ when computing a weighted average. It is a bad idea to sample it from a linear scale since the size of the window is highly sensitive on $\beta$ when $\beta$ is close to 1. Say that we want $\beta$ to be between 0.9 and 0.999, that is equivalent to requiring that $1-\beta=0.1$ and $1-\beta=0.001$. Therefore we can sample $\beta$ with `beta = 1 - 10**np.random.uniform(-2, -1)`.

### Tuning hyperparameters in practice

In practice we can either train a single model and tune it by changing its hyperparameters over time (that is a feasible approach with limited computational capacity), or we can train many models in parallel, finally choosing the best one at the end.